In [1]:
import os, time, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')


X_TEST_PATH = "/content/drive/MyDrive/DREAMERV_X.npy"
Y_TEST_PATH = "/content/drive/MyDrive/DREAMERV_y.npy"
CKPT_CNN    = "/content/drive/MyDrive/cnn_model.pth"
CKPT_LSTM   = "/content/drive/MyDrive/lstm_model.pth"

OUT_DIR = "/content/drive/MyDrive/Week10_Artifacts"
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE = "cpu"
np.random.seed(SEED); torch.manual_seed(SEED)
print("[INFO] Using DEVICE:", DEVICE)


Mounted at /content/drive
[INFO] Using DEVICE: cpu


In [2]:
X_test = np.load(X_TEST_PATH, allow_pickle=True)
y_test = np.load(Y_TEST_PATH, allow_pickle=True)
print(f"[INFO] X_test: {X_test.shape}, y_test: {y_test.shape}, labels: {np.unique(y_test)}")

def normalize_per_sample(X):
    # X: (N, 14, T)
    mean = X.mean(axis=2, keepdims=True)
    std  = X.std(axis=2, keepdims=True)
    std[std==0]=1
    return (X - mean) / std

X_test = normalize_per_sample(X_test).astype(np.float32)


[INFO] X_test: (170246, 14, 256), y_test: (170246,), labels: [0 1]


In [3]:
class EEG_CNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(14, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool  = nn.AdaptiveAvgPool1d(1)
        self.fc1   = nn.Linear(64, 32)
        self.fc2   = nn.Linear(32, num_classes)
    def forward(self, x):  # [B,14,T]
        x = self.conv1(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class EEG_LSTM(nn.Module):
    def __init__(self, input_size=14, hidden_size=64, num_layers=2, num_classes=2):
        super().__init__()
        self.hidden_size = hidden_size; self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1  = nn.Linear(hidden_size, 32)
        self.fc2  = nn.Linear(32, num_classes)
    def forward(self, x):  # [B,T,14]
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = F.relu(self.fc1(out[:,-1,:]))
        return self.fc2(out)

def load_cnn(path):
    m = EEG_CNN().to(DEVICE)
    m.load_state_dict(torch.load(path, map_location=DEVICE))
    m.eval()
    return m

def load_lstm(path):
    m = EEG_LSTM().to(DEVICE)
    m.load_state_dict(torch.load(path, map_location=DEVICE))
    m.eval()
    return m

cnn = load_cnn(CKPT_CNN)
lstm = load_lstm(CKPT_LSTM)
print("[INFO] Loaded CNN & LSTM checkpoints.")


[INFO] Loaded CNN & LSTM checkpoints.


In [4]:
def predict_logits(model, X, model_type="CNN", batch_size=256):
    N = X.shape[0]
    logits_all = []
    with torch.no_grad():
        for i in range(0, N, batch_size):
            xb = X[i:i+batch_size]
            if model_type=="CNN":
                xt = torch.from_numpy(xb).to(DEVICE)            # [B,14,T]
            else:
                xt = torch.from_numpy(xb).permute(0,2,1).to(DEVICE)  # [B,T,14]
            logits = model(xt).cpu().numpy()
            logits_all.append(logits)
    return np.vstack(logits_all)

def eval_metrics_from_logits(logits, y_true):
    probs = torch.softmax(torch.from_numpy(logits), dim=1).numpy()
    y_pred = probs.argmax(axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average="binary")
    try:
        auc = roc_auc_score(y_true, probs[:,1])
    except Exception:
        auc = np.nan
    cm  = confusion_matrix(y_true, y_pred)
    return acc, f1, auc, cm, probs, y_pred

def measure_latency_ms(model, X, model_type="CNN", reps=50, batch_size=256):
    # average end-to-end forward time per batch (ms)
    Xb = X[:batch_size]
    start = time.time()
    for _ in range(reps):
        _ = predict_logits(model, Xb, model_type=model_type, batch_size=batch_size)
    return (time.time() - start)/reps*1000.0


In [5]:
results = []

for name, model, mtype in [
    ("CNN_baseline",  cnn,  "CNN"),
    ("LSTM_baseline", lstm, "LSTM")
]:
    print(f"\n[INFO] Evaluating {name}")
    logits = predict_logits(model, X_test, model_type=mtype, batch_size=256)
    acc, f1, auc, cm, probs, y_pred = eval_metrics_from_logits(logits, y_test)
    lat_ms = measure_latency_ms(model, X_test, model_type=mtype, reps=20, batch_size=256)

    # Save confusion matrix plot
    fig, ax = plt.subplots(1,1, figsize=(3.5,3.5))
    ax.imshow(cm, cmap="Blues")
    for (i,j),v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha='center', va='center')
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(["Low","High"]); ax.set_yticklabels(["Low","High"])
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(f"{name} Confusion Matrix")
    cm_path = os.path.join(OUT_DIR, f"{name}_cm.png")
    plt.tight_layout(); plt.savefig(cm_path); plt.close()

    # Log row
    size_mb = os.path.getsize(CKPT_CNN if mtype=="CNN" else CKPT_LSTM)/(1024*1024)
    results.append([mtype, name, acc, f1, auc, lat_ms, round(size_mb,2), cm_path])

df_base = pd.DataFrame(results, columns=["Model","Variant","Accuracy","F1","ROC_AUC","Latency_ms","Size_MB","CM_PNG"])
df_base_path = os.path.join(OUT_DIR, "week10_eval_baselines.csv")
df_base.to_csv(df_base_path, index=False)
df_base



[INFO] Evaluating CNN_baseline

[INFO] Evaluating LSTM_baseline


,Model,Variant,Accuracy,F1,ROC_AUC,Latency_ms,Size_MB,CM_PNG
0,CNN,CNN_baseline,0.822721,0.772646,0.906422,18.716562,0.04,/content/drive/MyDrive/Week10_Artifacts/CNN_ba...
1,LSTM,LSTM_baseline,0.925719,0.912328,0.978691,154.305875,0.22,/content/drive/MyDrive/Week10_Artifacts/LSTM_b...


from matplotlib import pyplot as plt
df_base['Accuracy'].plot(kind='hist', bins=20, title='Accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_base['F1'].plot(kind='hist', bins=20, title='F1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_base['ROC_AUC'].plot(kind='hist', bins=20, title='ROC_AUC')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_base['Variant'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_base, x='Accuracy', y='Variant', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_base['CM_PNG'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_base, x='Accuracy', y='CM_PNG', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_base['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_base, x='F1', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [6]:
def add_gaussian_noise(X, sigma=0.05):
    return X + np.random.normal(0, sigma, size=X.shape).astype(np.float32)

def jitter_time(X, max_shift=3):
    # circular shift up to +/- max_shift along time axis independently per sample
    Xj = X.copy()
    N, C, T = X.shape
    shifts = np.random.randint(-max_shift, max_shift+1, size=N)
    for i,s in enumerate(shifts):
        Xj[i] = np.roll(Xj[i], s, axis=1)
    return Xj

def drop_random_channels(X, drop_prob=0.15):
    Xd = X.copy()
    N, C, T = X.shape
    mask = (np.random.rand(N, C) < drop_prob)
    for i in range(N):
        Xd[i][mask[i]] = 0.0
    return Xd

scenarios = {
    "clean":            lambda X: X,
    "noise_sigma_0.05": lambda X: add_gaussian_noise(X, 0.05),
    "jitter_3":         lambda X: jitter_time(X, 3),
    "channel_drop_0.15":lambda X: drop_random_channels(X, 0.15),
}

rows = []
for scen_name, transform in scenarios.items():
    Xs = transform(X_test)
    for name, model, mtype in [
        ("CNN_baseline",  cnn,  "CNN"),
        ("LSTM_baseline", lstm, "LSTM")
    ]:
        logits = predict_logits(model, Xs, model_type=mtype, batch_size=256)
        acc, f1, auc, cm, probs, y_pred = eval_metrics_from_logits(logits, y_test)
        rows.append([scen_name, mtype, name, acc, f1, auc])

df_stress = pd.DataFrame(rows, columns=["Scenario","Model","Variant","Accuracy","F1","ROC_AUC"])
df_stress_path = os.path.join(OUT_DIR, "week10_stress_results.csv")
df_stress.to_csv(df_stress_path, index=False)
df_stress


,Scenario,Model,Variant,Accuracy,F1,ROC_AUC
0,clean,CNN,CNN_baseline,0.822721,0.772646,0.906422
1,clean,LSTM,LSTM_baseline,0.925719,0.912328,0.978691
2,noise_sigma_0.05,CNN,CNN_baseline,0.821494,0.770652,0.905250
3,noise_sigma_0.05,LSTM,LSTM_baseline,0.916515,0.901706,0.974079
4,jitter_3,CNN,CNN_baseline,0.822545,0.772430,0.905646
5,jitter_3,LSTM,LSTM_baseline,0.914512,0.899371,0.973229
6,channel_drop_0.15,CNN,CNN_baseline,0.665907,0.619611,0.718865
7,channel_drop_0.15,LSTM,LSTM_baseline,0.669249,0.639412,0.728982


In [7]:
report = f"""# Week 10 – Final Stress Test Summary

**Device:** {DEVICE}
**Seed:** {SEED}
**Test set:** {X_test.shape[0]} samples, shape per sample: (14, {X_test.shape[2]})

## Baselines
{df_base.round(4).to_markdown(index=False)}

Confusion matrices saved:
- {df_base['CM_PNG'].iloc[0]}
- {df_base['CM_PNG'].iloc[1]}

## Stress Scenarios (Accuracy/F1/AUC)
{df_stress.round(4).to_markdown(index=False)}

### Notes
- Normalization: per-sample, per-channel (mean/std over time).
- Latency reported as average forward time per batch (ms).
- Scenarios: small Gaussian noise, small temporal jitter, random channel dropout.
"""

md_path = os.path.join(OUT_DIR, "Week10_System_Report.md")
with open(md_path, "w") as f:
    f.write(report)

print("[INFO] Wrote report →", md_path)


[INFO] Wrote report → /content/drive/MyDrive/Week10_Artifacts/Week10_System_Report.md
